In [1]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
data=pd.read_csv("train.csv")

In [4]:
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [5]:
data.shape

(20800, 5)

In [6]:
data.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [8]:
data.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [9]:
data.isnull()

,id,title,author,text,label
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
20795,False,False,False,False,False
20796,False,False,False,False,False
20797,False,False,False,False,False
20798,False,False,False,False,False


In [10]:
data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
20795    False
20796    False
20797    False
20798    False
20799    False
Length: 20800, dtype: bool

In [11]:
data.duplicated().sum()

0

### Cleaning Dataset

In [12]:
###Removing NaN Values from the dataset

In [13]:
data=data.dropna()
data.reset_index(inplace=True)

In [14]:
###Extracting Dependent and Independent variables from data

In [15]:
X = data.drop("label",axis=1)
y = data["label"]

In [16]:
###Checking The Shape of X and y

In [17]:
print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


In [18]:
###Preprocessing The Text 

In [19]:
messages=X.copy()
ps = PorterStemmer()
corpus = []

In [20]:
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]"," ",messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]  # (word not in stopwords) and stem(word)
  review = ' '.join(review)
  corpus.append(review)

### One Hot Encoding

In [21]:
vocab_size=5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 


In [22]:
###Applying Pad Sequences To Make Sentence Length Equal

In [23]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2100  302 3587]
 [   0    0    0 ... 1645 3156 4500]
 [   0    0    0 ...  886 1586 4036]
 ...
 [   0    0    0 ...  701 3091 3224]
 [   0    0    0 ...  739  935 1246]
 [   0    0    0 ...  872 2841 3813]]


### Creating Model for Train Set

In [24]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [25]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

### Splitting Dataset

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [27]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 6s 22ms/step - loss: 0.3397 - accuracy: 0.8426 - val_loss: 0.2002 - val_accuracy: 0.9122
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1403 - accuracy: 0.9450 - val_loss: 0.1981 - val_accuracy: 0.9147
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0973 - accuracy: 0.9646 - val_loss: 0.2296 - val_accuracy: 0.9153
Epoch 4/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0742 - accuracy: 0.9753 - val_loss: 0.2361 - val_accuracy: 0.9074
Epoch 5/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0497 - accuracy: 0.9841 - val_loss: 0.3136 - val_accuracy: 0.9155
Epoch 6/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0321 - accuracy: 0.9901 - val_loss: 0.3177 - val_accuracy: 0.9029
Epoch 7/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0206 - accuracy: 0.9936 - val_loss: 0.3827 - val_accuracy: 0.9032

### Performance Metrics And Accuracy

In [28]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

189/189 [==============================] - 1s 5ms/step


In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3073,  346],
       [ 199, 2417]], dtype=int64)

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.9096934548467275